# Star exploration

## Primer on stellar structure
In past weeks we have learned how to integrate various types of ODEs. We have used our own explicit forward integration method. We want to apply these skills to another example. 

### Explore stars with WENDI
Before we start to integrate our own stars let's explore some already calculated stellar models in [Wendi](http://wendi.nugridstars.org). These models were calculated with the [open source MESA stellar evolution code](<http://mesa.sourceforge.net>), [MESA code paper]( http://doi.org/10.1088/0067-0049/192/1/3).

__Homework__:

Mylib contains a star library with a function for the astrophysical ideal gas equation of state. One of the arguments is `mu` which is the mean molecular weight of the ideal gas. It is available as a quantity in the MESA profile data. The actual pressure profile calculated from the more realistic equation of state, including many non-ideal effects, is available from the `mesa_profile` instance as well.

1. Plot the pressure as a function of radius for both the 2 and 20Msun star at the time when the gravity at the surface is largest. (This happens to be the time when the star has contracted most and H-burning is ignited that will subsequently lead to expansion of the star and lowering of the surface gravity). For this you have to first use the `star_log` instance to do some meaning-full time evolution plots, likely as a function of model number. Then you have to create a `mesa_profile` instance to plot the pressure profile. 
2. For both cases get the temperature and density profiles at this point in time and determine the ideal gas pressure profile. By comparing the ideal gas pressure and the pressure from the stellar evolution model, determine how well the pressure in the two stellar models can be approximated by an ideal gas. 

### Stellar structure
Our goal is to determine the _mechanical_ structure of a pre-main sequence star. With _mechanical_ structure we mean that we want to know the pressure $P(r)$ as a function of radius $r$, as well as the density $\rho(r)$ and the enclosed mass $m(r)$.  

The pre-main sequence is the very first evoltuionary stage through which a star evolves just after it has formed. It is not yet burning nuclear fuel in the centre yet. When you explored the evolution of a star with WENDI you would have created a $\log T_{eff}$ - $\log L$ diagram ([Hertzsprung–Russell diagram](http://en.wikipedia.org/wiki/Hertzsprung-Russell_diagram)). The pre-main sequence is the beginning of the track at low temperature $T_{eff}$ (astronomers denote some appropriately defined average surface temperature with the _effective_ temperature $T_{eff}$) and initially high and then decreasing luminosity $L$ (the luminosity or stellar luminosity is the photon energy flux $F$ at all wavelength together times the surface area of the star: $L_\star = 4\pi R_\star^2 F$).

We chose pre-main sequence stars because they are known to be fully convective, i.e. unstable against convective turnover, throughout from the center to the surface. This can be understood as a consequence of the low stellar temperature at which the stellar plasma is more opaque compared to hotter stars. Energy that has to leave the star as it contracts can not be entirely transported by radiation, and therefore convective transport, i.e. the tranport of heat by fluid elements that are moving radially and carrying their heat along with them, is dominating the energy transport. In that case it can be easily shown (see e.g. Kippenhahn & Weigert, Stellar Structure and Evolution) that for an ideal gas the pressure and density are related by a so-called _polytropic_ relation
$$
P = K \rho^\gamma
$$
where $K$ is a constant and $\gamma = \gamma_{ad}=\frac{5}{3}$ is the adiabatic exponent. 

What is furtheron the physics that determines the structure of the star? The stellar plasma has to obey the conservation law of mass and momentum (because we specifically are interested only the _mechanical_ structure we do not have to deal with energy conservation). 

We assume that our star is to very good approximation spherical. This assumption is supported by the large gravity of stars that enforce sphericity effectively. 

In spherical symmetry the conservation of mass can be easily expressed by just stating how much mass $dm$ is contained in a spherical shell of size $dr$ at radius $r$ where the density is $\rho$:

$$
\frac{dm}{dr} = 4 \pi r^2 \rho(r)
$$

We further assume that the star is in hydro-static equilibrium. That means that global, bulk velocities are essentially zero (convective motions that average to a zero mass flux on spheres are allowed and their effect of mixing heat and species needs to be _somehow_ taken into effect, but we do not need to worry about that here). In this approximation the conservation of momentum states that the pressure gradient accross a small radial shell $dr$ (i.e. the pressure difference $dP$) is equal to the gravitational force acting on the mass in that shell:

$$
\frac{dP}{dr} = -g \rho = -\frac{G*m(r)}{r^2} \rho(r)
$$

where $G$ is the gravitational constant (that I added as `grav_const` to the `mylib/stars/star_utils.py` module). 

We have now three unknown quantities: $\rho(r)$, $m(r)$ and $P(r)$ and we have three equations:

* two differential equations - the conservation laws of mass and momentum
* the polytropic relation $P(\rho)$ that holds if we know that the layer in the star is unstable against convection, i.e. adiabatic. 

We are therefore able to integrate a structure for a given boundary conditions. These are for the center $m_c = 0$ and $P_c$ and $\rho_c$ appropriately choosen. In a dedicated stellar astrophysics class we would learn how to estimate these central values. It is not difficult! But here, let's skip that, and just take central values from the pre-calculated stellar evolution models (all units are in cgs!!). 

In [ ]:
# get session ready:
%pylab  nbagg                                                                            

In [ ]:
import sys
sys.path.append('../mypylib')
from stars import star_utils as su

In order to use the available range of number representation it is often useful to transform the problem from cgs or SI units to appropriately choosen code units. The goal is that all quantities that the code works with have values of order unity, or as close as possible. We do not routinely perform floating point operations with large exponents. Why?

In [ ]:
# introduce code units:                                                               
# one unit of density: g_cu * cm_cu**-3 = 0.1691355                                   
# one length unit:  1R_sun = 6.955e+10 cm                                             
# one unit of mass: 1M_sun = 1.9891e+33 g                                             
# one unit of time: 1yr    = 3.14e7s    

In [ ]:
L_cu = su.rsun_cm    # length in code unit                                            
cm_cu = 1./L_cu
M_cu = su.msun_g
g_cu = 1./M_cu
T_cu = 3.14e7
s_cu = 1/T_cu

In [ ]:
# G in units of 'cm^3 g^-1 s^-2'  
G_cu = su.grav_const * cm_cu**3 * g_cu**-1 * s_cu**-2
# one unit of pressure: [cgs: g cm^-1 s^-2] -> Msun Rsun^-1 yr^-2                     

In [21]:
# central values from the WENDI platform:
rho_c = 0.0010887436239364599    # in cgs                                             
rho_c = rho_c * g_cu * cm_cu**-3 # cu                                                 
P_c = 89386992764.462601         # in cgs                                             
P_c = P_c * g_cu * cm_cu**-1 * s_cu**-2 # cu                                          
print(rho_c, P_c)

# p2.get('radius')[-1],p2.get('mass')[-1]                                             
r_c = 0.058674868915132176   # in L_cu = R_sun                                        
m_c = 1.5612859321774683e-07 # in M_cu = M_sun                                        

(0.00018414520313142182, 3081.584916444496)


Next we need to code the constituting equations. Let's start with the P-$\rho$ relation:

In [ ]:
# EOS:                                                                                
gamma_ad = 5./3.
K_ad = P_c/(rho_c**gamma_ad)
K    = K_ad
print(K)

def rho(p):   # polytropic EOS                                                        
        return (p/K)**(1./gamma_ad)

In [ ]:
# unit tester EOS:                                                                    
print(rho(P_c), rho_c)


Now we need the RHS of the PDEs:

In [ ]:
# RHS of system of ODEs                                                               
def f_rhs(y,r):
    dm_dr = 4.*pi*(r**2)*rho(y[1])
    dp_dr = -rho(y[1])*G_cu*y[0]/(r**2)
    return [dm_dr,dp_dr]

We also need the boundary condition at the center:

In [ ]:
# initial conditions                                                                  
V0  = 4./3*pi*r_c**3
m_0 = rho_c*V0
print(m_0,m_c)

And we need the actual Euler step routine:

In [ ]:
def int_eul(nsteps,r_c,dr,y0):
    r  = r_c
    dr = dr
    y  = array(y0)
    pp = []; mp = []; rp = []
    for i in range(nsteps):
        rhs = f_rhs(y,r)
        y += array(rhs)*dr
        r += dr
        mp.append(y[0])
        pp.append(y[1])
        rp.append(r)
    return mp, pp, rp

Finally, let's run it and plot the result:

In [ ]:
y0=[m_0,P_c]
dr = 0.01
steps = 2300
mass,pressure,radius = int_eul(steps,r_c,dr,y0)

In [ ]:
plot(mass,radius)
xlabel('mass/[Msun]')
ylabel('radius/[Rsun]')

***Homework:*** 
Instead of the explicit Euler step method modify the code so that you can use the `integrate.odeint` solver from `scipy`. Below are some code fragments that can get you started.

In [ ]:
from scipy import integrate
# integrate.odeint?

In [ ]:
f_rhs(y0,r[0])

In [ ]:
Y=integrate.odeint(f,y0,r)

In [ ]:
m_poly = Y.transpose()[0]
p_poly = Y.transpose()[1]